In [51]:
from libertem.common.array_backends import (
    CPU_BACKENDS, CUDA_BACKENDS, CUPY_BACKENDS, NUMPY, CUPY, CUDA, SPARSE_GCXS, SCIPY_COO, CUPY_SCIPY_COO, BACKENDS
)
from collections import OrderedDict, defaultdict

In [87]:
class UDF1:
    def get_backends(self):
        # Preference for dense, only ndarray
        return (NUMPY, CUPY, SPARSE_GCXS)

class UDF2:
    def get_backends(self):
        # Fallback, legacy default
        return (NUMPY, )

class UDF3:
    def get_backends(self):
        # Preference for sparse, duplicate backend specified
        return (SCIPY_COO, CUPY_SCIPY_COO, CUPY, NUMPY, NUMPY)

class UDF4:
    def get_backends(self):
        # Return single value instead of iterable
        return CUPY_SCIPY_COO
    
class UDF5:
    def get_backends(self):
        # Fallback, legacy default
        return (CUDA, CUPY)

In [95]:
class DS1:
    @property
    def array_backends(self):
        return (NUMPY, CUDA)

class DS2:
    @property
    def array_backends(self):
        # Supposed event dataset
        return (SCIPY_COO, CUPY_SCIPY_COO)

class DS3:
    @property
    def array_backends(self):
        return (NUMPY, CUDA, CUPY)

In [96]:
def _canonical_backends(backends):
    if isinstance(backends, str):
        return (backends, )
    else:
        return backends

In [99]:
def execution_plan(ds, device_class, udfs, available_backends=BACKENDS):
    remaining = list(udfs)
    execution_plan = OrderedDict()
    if device_class == 'cuda':
        native_backends = available_backends.intersection(CUDA_BACKENDS)
    elif device_class == 'cpu':
        native_backends = available_backends(CPU_BACKENDS)
    else:
        raise ValueError(f"Unknown device class{device_class}, allowed are 'cuda' and 'cpu'.")
        
    aggregate_udf_aversion = defaultdict(lambda: 0.)
    for udf in remaining:
        backends = _canonical_backends(udf.get_backends())
        for i, b in enumerate(backends):
            aggregate_udf_aversion[b] += i / len(backends)
    
    backend_popularity = sorted(aggregate_udf_aversion.keys(), key=lambda b: aggregate_udf_aversion[b])
    
    def find_popular(remaining, restrict_to, preference_order=None):
        '''
        Parameters
        ----------
        
        preference_order: Order of preference for tie breaking if several formats can work for the same number of UDFs
        '''
        if preference_order is None:
            preference_order = ( )
        popular = defaultdict(OrderedDict)
        for udf in remaining:
            for b in _canonical_backends(udf.get_backends()):
                if b in restrict_to:
                    # Use dict with value True to make sure UDFs are not counted double
                    popular[b][udf] = True
        most_popular = sorted(popular.keys(), key=lambda b: len(popular[b]), reverse=True)
        if most_popular:
            max_len = len(popular[most_popular[0]])
            most_popular = [b for b in most_popular if len(popular[b]) >= max_len]
            if len(most_popular) > 1:
                for b in preference_order:
                    if b in most_popular:
                        return b, list(popular[b].keys())
            b = most_popular[0]
            return b, list(popular[b].keys())
        else:
            return None, None
    
    def apply(backend, udfs):
        if backend is not None:
            execution_plan[backend] = udfs
            for udf in udfs:
                remaining.remove(udf)
                
    # First assign UDFs that work without conversion
    # with a tile format native to dataset and device class
    apply(*find_popular(
        remaining,
        restrict_to=native_backends.intersection(ds.array_backends),
        preference_order=ds.array_backends
    ))
    # Then assign UDFs that work with an array format that matches the devie class
    # That means arrays will be converted to allow execution on GPU or CPU even if the
    # dataset can't deliver that natively. That is the case for the dataset default NumPy
    while True:
        backend, selected_udfs = find_popular(
            remaining,
            restrict_to=native_backends,
            preference_order=backend_popularity
        )
        if backend is not None:
            apply(backend, selected_udfs)
        else:
            break
    # Assign UDFs that work with a tile fomat native to the dataset
    apply(*find_popular(
        remaining,
        restrict_to=available_backends.intersection(ds.array_backends),
        preference_order=ds.array_backends
    ))
    # Finally assign UDFs to array formats even if they don't match the
    # dataset or device class. This includes CPU fallback on GPU workers
    while True:
        backend, selected_udfs = find_popular(
            remaining,
            restrict_to=available_backends,
            preference_order=backend_popularity
        )
        if backend is not None:
            apply(backend, selected_udfs)
        else:
            break
    # Now we should have covered all
    if remaining:
        for r in remaining:
            for b in _canonical_backends(r.get_backends()):
                if b not in BACKENDS:
                    raise ValueError(f"UDF {r} returned invalid backend {b}.")
        raise RuntimeError(f"Could not find backends for remaining {remaining}.")
        
    all_udfs = set()
    for b, selected_udfs in execution_plan.items():
        for udf in selected_udfs:
            assert b in _canonical_backends(udf.get_backends())
            assert b in available_backends
            assert udf not in all_udfs
            all_udfs.add(udf)
    assert set(udfs) == all_udfs
            
    return execution_plan
        

In [101]:
execution_plan(DS3(), 'cuda', (UDF1(), UDF2(), UDF3(), UDF5()), CPU_BACKENDS.union({CUDA}))

OrderedDict([('cuda', [<__main__.UDF5 at 0x1f95f71bca0>]),
             ('numpy',
               <__main__.UDF3 at 0x1f95f71b220>])])

In [27]:
l = [1, 2, 3]
ll = list(l)
ll.remove(3)
l

[1, 2, 3]

In [1]:
np

NameError: name 'np' is not defined

In [2]:
np.matrix

NameError: name 'np' is not defined